# Train Chow-Liu Tree

In [6]:
import numpy as np
import copy
import pickle
import sys


class chow_liu:
    def pm(self, dataset, m, n):
        pxy = np.zeros((n, n, 4))
        for i in range(n):
            temp_dataset = dataset[dataset[:, i] == 0]
            for j in range(n):
                if pxy[i, j, 0] == 0:
                    pxy[i, j, 0] = (temp_dataset[temp_dataset[:, j] == 0].shape[0]+1)/(m+4)
                if pxy[j, i, 0] == 0:
                    pxy[j, i, 0] = pxy[i, j, 0]
                if pxy[i, j, 1] == 0:
                    pxy[i, j, 1] = (temp_dataset[temp_dataset[:, j] == 1].shape[0]+1)/(m+4)
                if pxy[j, i, 2] == 0:
                    pxy[j, i, 2] = pxy[i, j, 1]

            temp_dataset = dataset[dataset[:, i] == 1]
            for j in range(n):
                if pxy[i, j, 2] == 0:
                    pxy[i, j, 2] = (temp_dataset[temp_dataset[:, j] == 0].shape[0]+1)/(m+4)
                if pxy[j, i, 1] == 0:
                    pxy[j, i, 1] = pxy[i, j, 2]
                if pxy[i, j, 3] == 0:
                    pxy[i, j, 3] = (temp_dataset[temp_dataset[:, j] == 1].shape[0]+1)/(m+4)
                if pxy[j, i, 3] == 0:
                    pxy[j, i, 3] = pxy[i, j, 3]
        return pxy


    def mi(self, pxy, n):
        ixy = np.zeros((n, n))
        for i in range(n):
            for j in range(n):
                if i < j:
                    ixy[i, j] = pxy[i, j, 0]*np.log(pxy[i, j, 0]/(pxy[i, i, 0]*pxy[j, j, 0])) + pxy[i, j, 1]*np.log(pxy[i, j, 1]/(pxy[i, i, 0]*pxy[j, j, 3])) + \
                        pxy[i, j, 2]*np.log(pxy[i, j, 2]/(pxy[i, i, 3]*pxy[j, j, 0])) + \
                        pxy[i, j, 3]*np.log(pxy[i, j, 3]/(pxy[i, i, 3]*pxy[j, j, 3]))
        return ixy


    def cm(self, dataset, tr, cols):
        cxy = np.zeros((len(tr), cols))
        for idx, node in enumerate(tr):
            i, j = node
            cxy[idx] = [dataset[(dataset[:, i] == 0) & (dataset[:, j] == 0)].shape[0], dataset[(dataset[:, i] == 0) & (dataset[:, j] == 1)].shape[0], dataset[(
                dataset[:, i] == 1) & (dataset[:, j] == 0)].shape[0], dataset[(dataset[:, i] == 1) & (dataset[:, j] == 1)].shape[0]]
        return cxy


    def mtr(self, ls, ntr, root):
        for node in [item for item in ls if root in item]:
            if node[0] == root:
                ntr.append(node)
                ls.remove(node)
                self.mtr(ls, ntr, node[1])
            else:
                ntr.append((node[1], node[0]))
                ls.remove(node)
                self.mtr(ls, ntr, node[0])

    def dt(self, ewts, prnt=False):
        ewts_cp = copy.deepcopy(ewts)
        edges = [np.unravel_index(np.argmax(ewts_cp), ewts_cp.shape)]
        visited = [[edges[-1][0], edges[-1][1]]]
        ewts_cp[edges[-1]] = 0
        while(len(edges) < ewts.shape[0]-1):
            i = j = -1
            edge = np.unravel_index(np.argmax(ewts_cp), ewts_cp.shape)
            for bag in range(len(visited)):
                if edge[0] in visited[bag]:
                    i = bag
                if edge[1] in visited[bag]:
                    j = bag
            if i == -1 and j != -1:
                edges.append(edge)
                visited[j].append(edge[0])
            elif i != -1 and j == -1:
                edges.append(edge)
                visited[i].append(edge[1])
            elif i == -1 and j == -1:
                edges.append(edge)
                visited.append([edge[0], edge[1]])
            elif i != -1 and j != -1 and i != j:
                edges.append(edge)
                visited[i] += visited[j]
                visited.remove(visited[j])
            elif i == j != -1:
                pass
            else:
                print("Discarded in else", edge)
            ewts_cp[edge] = 0

        ntr = []
        self.mtr(edges, ntr, edges[0][0])

        return ntr

    def train(self, dataset, m, n):
        pxy = chow_liu.pm(dataset, m, n)
        ixy = chow_liu.mi(pxy, n)
        tr = chow_liu.dt(ixy, False)
        tr = [(tr[0][0], tr[0][0])] + tr
        cp = np.zeros((len(tr), pxy.shape[2]))
        for idx, node in enumerate(tr):
            if node[0] == node[1]:
                cp[idx] = np.log(pxy[node[0], node[1], :])
            else:
                cp[idx] = np.log(np.hstack(((pxy[node[0], node[1], :2]/pxy[node[0],node[0], 0]), (pxy[node[0], node[1], 2:]/pxy[node[0], node[0], 3]))))

        return tr, pxy, cp

    def generate_model(self, dataset, m, n):
        model = chow_liu.train(dataset, m ,n)
        f = open('modelclt_book.pkl', 'wb')
        pickle.dump(model, f, -1)
        f.close()    
    
def dataset_load(file, file_2=None):
    dataset_1 = np.loadtxt(file, delimiter=",", dtype=int)
    if file_2:
        dataset_2 = np.loadtxt(file, delimiter=",", dtype=int)
        dataset = np.vstack((dataset_1, dataset_2))
    else:
        dataset = dataset_1
    return dataset, dataset.shape[0], dataset.shape[1]

chow_liu = chow_liu()
dataset, m, n = dataset_load("book.ts.data")
chow_liu.generate_model(dataset,m,n)

# Testing File

In [2]:
test = np.loadtxt("nltcs.valid.data", delimiter=",", dtype=int)
f = open('modelclt_nltcs.pkl', 'rb')
tr, pxy, cp = pickle.load(f)     
f.close()  
cxy = chow_liu.cm(test, tr, pxy.shape[2])
print("ALL:", np.sum(cxy*cp)/test.shape[0])

ALL: -6.716787490224922


In [4]:
test = np.loadtxt("plants.valid.data", delimiter=",", dtype=int)
f = open('modelclt_plants.pkl', 'rb')
tr, pxy, cp = pickle.load(f)     
f.close()  
cxy = chow_liu.cm(test, tr, pxy.shape[2])
print("ALL:", np.sum(cxy*cp)/test.shape[0])

ALL: -16.509888925591692


In [7]:
test = np.loadtxt("book.valid.data", delimiter=",", dtype=int)
f = open('modelclt_book.pkl', 'rb')
tr, pxy, cp = pickle.load(f)     
f.close()  
cxy = chow_liu.cm(test, tr, pxy.shape[2])
print("ALL:", np.sum(cxy*cp)/test.shape[0])

ALL: -36.634123975145
